In [4]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv()

True

In [2]:
from dataclasses import dataclass

@dataclass
class ColourContext:
    favourite_colour: str = "blue"
    least_favourite_colour: str = "yellow"

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="us.meta.llama4-maverick-17b-instruct-v1:0",  # Nota el prefijo "us."
    region_name="us-east-1",
    beta_use_converse_api=True,
    model_kwargs={
        "temperature": 0.5,
        "max_tokens": 2048,
        "top_p": 0.9,
    }
)


from langchain.agents import create_agent

agent = create_agent(
    model="nova",
    context_schema=ColourContext  
)

In [13]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?")]},
    context=ColourContext()
)

In [14]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?', additional_kwargs={}, response_metadata={}, id='e3f37906-c3a9-4cda-9c00-4397b9b67470'),
              AIMessage(content="\n\nYou haven't told me your favourite colour, so I don't have any information about it. If you'd like to share, I'd be happy to remember it for our conversation!", additional_kwargs={'usage': {'prompt_tokens': 16, 'completion_tokens': 35, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 51}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 16, 'completion_tokens': 35, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 51}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, id='lc_run--019b4e35-295a-7bf3-b1f6-12b51c377ebb-0'

## Accessing Context

In [25]:
from langchain.tools import tool, ToolRuntime

@tool
def get_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the favourite colour of the user"""
    return runtime.context.favourite_colour

@tool
def get_least_favourite_colour(runtime: ToolRuntime) -> str:
    """Get the least favourite colour of the user"""
    return runtime.context.least_favourite_colour

In [26]:
from langchain.agents import create_agent
agent = create_agent(
    model=llm,
    tools=[get_favourite_colour, get_least_favourite_colour],
    context_schema=ColourContext
)

In [27]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour?. Could you read the @tool.")]},
    context=ColourContext()
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour?. Could you read the @tool.', additional_kwargs={}, response_metadata={}, id='309635bc-e3e5-493e-8f00-6cafaf5ba4eb'),
              AIMessage(content='\n\nI don\'t have have access to any external tools or data about you, so I\'m unable to read any "@tool" or know your personal preferences, including your favourite colour. However, I can try to help you discover or decide on a favourite colour if you\'d like to discuss it!', additional_kwargs={'usage': {'prompt_tokens': 23, 'completion_tokens': 58, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 81}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 23, 'completion_tokens': 58, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 81}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 

In [30]:
response = agent.invoke(
    {"messages": [HumanMessage(content="What is my favourite colour? ")]},
    context=ColourContext(favourite_colour="green")
)

pprint(response)

{'messages': [HumanMessage(content='What is my favourite colour? ', additional_kwargs={}, response_metadata={}, id='689b8424-aec5-4cdb-bd88-0b719b54abde'),
              AIMessage(content="\n\nYou haven't told me your favourite colour, so I don't have any information about it. If you'd like to share, I'd be happy to remember it for our conversation!", additional_kwargs={'usage': {'prompt_tokens': 17, 'completion_tokens': 35, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 52}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 17, 'completion_tokens': 35, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 52}, 'stop_reason': 'stop', 'thinking': {}, 'model_id': 'us.meta.llama4-maverick-17b-instruct-v1:0', 'model_provider': 'bedrock', 'model_name': 'us.meta.llama4-maverick-17b-instruct-v1:0'}, id='lc_run--019b4e3f-8e9d-7772-bd94-c5f342877b7e-0